<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/04%20Deep%20Learning/notebooks/02-MLP-Clasificacion-Binaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción de Diabetes con MLP  

<h2>Introducción</h2>

La diabetes es una enfermedad crónica que afecta a millones de personas en el mundo, y su detección temprana es crucial para prevenir complicaciones graves. El *Diabetes Prediction Dataset* proporciona información médica y demográfica de pacientes, incluyendo variables como edad, género, índice de masa corporal (IMC), niveles de HbA1c, glucosa en sangre, historial de hipertensión, enfermedades cardíacas y hábitos de tabaquismo. Este conjunto de datos permite desarrollar modelos de aprendizaje automático para predecir el diagnóstico de diabetes (clasificación binaria: positivo/negativo), apoyando a profesionales de la salud en la identificación de pacientes en riesgo y en la personalización de tratamientos.  

Sin embargo, el dataset presenta desafíos clave:  
1. **Desbalance de clases**: Solo el **8.5%** de las muestras corresponden a casos positivos (diabetes), lo que puede sesgar el entrenamiento de modelos tradicionales.  
2. **Variables categóricas**: Género y historial de tabaquismo requieren codificación adecuada (ej.: one-hot encoding).  
3. **Tamaño del dataset**: Con **100,000 muestras**, es necesario evaluar si una arquitectura MLP puede capturar patrones complejos sin sobreajuste.  

<h2>Objetivos</h2>

1. **Diseñar arquitecturas MLP** eficientes para clasificación binaria, explorando diferentes arquitecturas (capas ocultas, neuronas, funciones de activación).  
2. **Manejar el desbalance de clases** mediante:
   - Reponderación con `class_weight` en Keras (ej.: `{0: 1.0, 1: 12.0}`).  
   - Evaluación con métricas robustas (**Recall, F1-Score, AUC-ROC**) en lugar de *accuracy*.
3. **Preprocesar variables categóricas** y numéricas para garantizar su compatibilidad con la MLP.
4. **Validar la escalabilidad** de la MLP con datasets de tamaño medio (100k muestras), monitoreando tiempos de entrenamiento y uso de recursos.

Este proyecto busca demostrar cómo una red neuronal básica (MLP) puede ser competitiva en problemas médicos reales, incluso con datos desbalanceados y heterogéneos, proporcionando un *baseline* para comparaciones con modelos más complejos (ej.: XGBoost, Redes Convolucionales).  

https://www.kaggle.com/datasets/iammustafatz/diabetes-prediction-dataset

Re-instalamos keras para evitar el warning de la importación

In [ ]:
!pip install -qq keras

Descargamos directamente de kaggle el dataset

In [ ]:
import pandas as pd
import os
#--------- Este código lo copiamos desde Kaggle -----------
import kagglehub

# Download latest version
path = kagglehub.dataset_download("iammustafatz/diabetes-prediction-dataset")

print("Path to dataset files:", path)
#-----------------------------------------------------------

path = os.path.join(path, 'diabetes_prediction_dataset.csv') # Unimos la ruta de descarga con el nombre de archivo
df = pd.read_csv(path) # Leemos el dataset
df

🟢 Veamos que no hay datos faltantes

In [ ]:
print(df['gender'].unique())
print(df['smoking_history'].unique())

In [ ]:
df.isna().sum()

🔴 Haz codificación *one-hot* para las variables categóricas. No olvides usar `dtype=int` y `drop_first=True`.

In [ ]:
df =

🟢 Dividamos el conjunto de entrenamiento en *train/validation/test*

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('diabetes', axis=1).values
y = df['diabetes'].values

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.15,
                                                    stratify = y,
                                                    random_state=423)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=0.2,
                                                  stratify = y_train,
                                                  random_state=423)

print(X_train.shape, X_val.shape, X_test.shape)

🟢 Visualizar el balance de clases

In [ ]:
from seaborn import countplot
import matplotlib.pyplot as plt

ratio = y[y == 1].shape[0] / y.shape[0]

plt.figure()
plt.title(f'Balance de clases\nClase positiva {ratio} del total')
countplot(data=df,x='diabetes')
plt.show()

🟢 Probemos esta estrategia para el desbalanceo de clases. Consiste en calcular pesos para las clases, de acuerdo al desbalanceo. Posteriormente se usan esos pesos para ponderar las métricas durante el entrenamiento.

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight('balanced',
                                     classes=np.unique(y_train),
                                     y=y_train)
class_weights = {0: class_weights[0], 1: class_weights[1]}
class_weights

🔴 Haz re-escalamiento `MinMaxScaler`. No olvides evitar el *data leakage*

In [ ]:
from sklearn.preprocessing import MinMaxScaler



🔴 Define una red neuronal con la clase `Sequential` de Keras para este problema

* Define la capa de entrada adaptada al número de features
* Define el número de capas ocultas y la cantidad de neuronas en cada una de ellas, usa la activación `relu`.
* Define la capa de salida adecuada para la clasificación binaria.
* Compila el modelo con la métrica Recall (por el desbalaceo de clases), define el compilador y la función de perdida adecuada para la clasificación binaria.

<br>

----
<br>

Una guía general sobre funciones de perdida y métricas

<br>

| Aplicación                     | Función de Pérdida (Loss)          | Métrica usual           | Última capa (output layer)          |
|---------------------------------|------------------------------------|-------------------------|-------------------------------------|
| Clasificación binaria          | `binary_crossentropy`              | `accuracy`              | `Dense(1, activation='sigmoid')`    |
| Clasificación multiclase       | `categorical_crossentropy`         | `accuracy`              | `Dense(num_clases, activation='softmax')` |
| Regresión (un valor)           | `mean_squared_error` (MSE)         | `mse` o `mae`           | `Dense(1, activation='linear')`     |
| Regresión (múltiples valores)  | `mean_squared_error` (MSE)         | `mse` o `mae`           | `Dense(num_valores, activation='linear')` |

<br>

---
<br>

Una guía general sobre optimizadores:

<br>

| Optimizador  | Ventajas                             | Casos de Uso Típicos         | Parámetros Clave               |
|--------------|--------------------------------------|------------------------------|--------------------------------|
| **Adam**     | Convergencia rápida, adaptable      | Default para MLPs, CNN, RNN  | `lr`  |
| **SGD**      | Mayor control, estable con momentum | Problemas convexos, fine-tuning | `lr`, `momentum`      |
| **RMSprop**  | Bueno para datos ruidosos           | RNNs, problemas inestables    | `lr`, `rho`          |

<br>

---

<br>

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Input

#----- Completa --------
model = ...


#-----------------------

model.summary()

🔴 Entrena el modelo

* Elige, al menos 20 épocas.
* Usa el hiperparámetro `class_weight=class_weights`.
* Usa el hiperparámetro `batch_size=256` o `batch_size=32`. Observa la diferencia en tiempo de entrenamiento.
* Usa el conjunto de validación como `validation_data`.

Recuerda almancenar el registro de entrenamiento en la variable `history`.

In [ ]:
history = ...

🟢 Graficamos las curvas de entrenamiento

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(15, 5))
axs[1].plot(history.history['recall'])
axs[1].plot(history.history['val_recall'])
axs[1].set_title('Model Recall')
axs[1].set_ylabel('Recall')
axs[1].set_xlabel('Epoch')
axs[1].legend(['Train', 'Validation'], loc='upper left')
axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].set_title('Model Loss')
axs[0].set_ylabel('Loss')
axs[0].set_xlabel('Epoch')
axs[0].legend(['Train', 'Validation'], loc='upper left')
fig.show()

🔴 Obten las predicciones del modelo para el conjunto de prueba. Impríme el arreglo, ¿qué interpretación tienen los valores que observas?

In [ ]:
y_pred =

🟢 Obten las predicciones como valores de clase

In [ ]:
y_pred = (y_pred > 0.5).astype(int)

🔴 Mide el rendimiento del modelo en el conjunto de prueba. Reporta las métricas:

* Accuracy
* Recall
* Precision
* F1 Score

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


🟢 Muestra la matriz de confusión

In [ ]:
from sklearn.metrics import confusion_matrix
from seaborn import heatmap
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
plt.show()

🔴 **OPCIONAL** Como comparación, realiza la misma tarea de clasificación binaria entrenando un clasificador de Machine Learning clásico (de scikit-learn).

Reporta la mismas métricas y la matriz de confusión.

* ¿Cuál tuvo mejor rendimiento?
* ¿Cuál tardo más en entrenarse?